In [ ]:
#conda install -n crewai_salary_optimization ipykernel --update-deps --force-reinstall


In [ ]:
import os
import gc
from crewai import Agent, Task, Crew
import pandas as pd
from langchain_openai import ChatOpenAI



# Clear existing DataFrames
if 'df' in globals():
    del globals()['df']
if 'employee_data' in globals():
    del globals()['employee_data']

# Force garbage collection
gc.collect()



# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "KEY"  # Replace with your actual key

In [ ]:

# Sample data
employee_data_0 = {
    'name': ['Alex', 'ُSunil', 'Martin', 'Jack', 'Ryan', 'John'],
    'salary': [10, 20, 100, 2, 10, 20],
    'location': ['USA', 'Canada', 'France', 'England', 'India', 'Canada'],
    'job': ['Data', 'Web', 'Data', 'Web', 'Data', 'Web']
}

# Convert to DataFrame
df = pd.DataFrame(employee_data_0)

In [ ]:
import pandas as pd
import random

# Expanded lists for generation
names = ['Mehdi', 'Sunil', 'Martin', 'Alex', 'Ryan', 'John', 'Sarah', 'Emma', 'Liam', 'Olivia',
         'Noah', 'Ava', 'William', 'Sophia', 'James', 'Isabella', 'Oliver', 'Mia', 'Benjamin', 'Charlotte',
         'Elijah', 'Amelia', 'Lucas', 'Harper', 'Mason', 'Evelyn', 'Logan', 'Abigail', 'Alexander', 'Emily',
         'Ethan', 'Elizabeth', 'Jacob', 'Mila', 'Michael', 'Ella', 'Daniel', 'Avery', 'Henry', 'Sofia',
         'Jackson', 'Camila', 'Sebastian', 'Aria', 'Aiden', 'Scarlett', 'Matthew', 'Victoria', 'Samuel', 'Madison',
         'David', 'Luna', 'Joseph', 'Grace', 'Carter', 'Chloe', 'Owen', 'Penelope', 'Wyatt', 'Riley',
         'Jayden', 'Nora', 'Dylan', 'Lily', 'Luke', 'Hannah', 'Gabriel', 'Zoey', 'Isaac', 'Layla',
         'Anthony', 'Zoe', 'Julian', 'Natalie', 'Andrew', 'Addison', 'Levi', 'Lillian', 'Nathan', 'Brooklyn',
         'Caleb', 'Audrey', 'Jack', 'Savannah', 'Lincoln', 'Claire', 'Mateo', 'Skylar', 'Leo', 'Bella',
         'Hunter', 'Anna', 'Grayson', 'Ellie', 'Connor', 'Stella', 'Jaxon', 'Sadie', 'Isaiah', 'Paisley',
         'Jose', 'Emilia', 'Thomas', 'Violet']

locations = ['Canada', 'USA', 'France', 'England', 'India', 'Germany', 'Australia', 'Japan', 
             'Brazil', 'Spain', 'Italy', 'Mexico', 'South Korea', 'Netherlands', 'Sweden']

jobs = ['Data Analyst', 'Web Developer', 'Software Engineer', 'Project Manager', 'UX Designer',
        'DevOps Engineer', 'Business Analyst', 'QA Engineer', 'System Administrator', 'Mobile Developer',
        'Data Scientist', 'Network Engineer', 'Product Manager', 'Graphic Designer', 'Cybersecurity Analyst']

# Generate data for 100 employees
employee_data = {
    'name': random.choices(names, k=100),
    'salary': [random.randint(30, 150) for _ in range(100)],  # Salaries in thousands
    'location': random.choices(locations, k=100),
    'job': random.choices(jobs, k=100)
}

# Convert to DataFrame
df = pd.DataFrame(employee_data)

# Display first few rows and total count
print(df.head())
print(f"\nTotal number of employees: {len(df)}")

In [ ]:
# Shared LLM configuration
llm = ChatOpenAI(model="gpt-4-turbo")

# 1. Data Analyst Agent
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="Analyze current workforce distribution and salary costs",
    backstory="""You're an expert in workforce analytics with years of experience 
    in organizational data analysis. You identify patterns and key metrics.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 2. Cost Optimizer Agent
cost_optimizer = Agent(
    role="Cost Optimization Specialist",
    goal="Identify ways to reduce salary costs with minimal impact",
    backstory="""You're a financial efficiency expert who specializes in 
    workforce cost optimization while maintaining operational capacity.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 3. Geo Balance Specialist
geo_specialist = Agent(
    role="Geographic Distribution Manager",
    goal="Maintain balanced team distribution across locations",
    backstory="""You ensure companies maintain their global presence and 
    comply with local employment requirements.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# 4. Job Function Specialist
role_specialist = Agent(
    role="Job Function Coordinator",
    goal="Preserve necessary roles in each location",
    backstory="""You understand organizational design and ensure critical 
    functions are properly staffed across all locations.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [ ]:


analysis_task = Task(
    description=f"""Analyze the current {df} workforce distribution including:
    - Locations: ONLY {df['location'].unique().tolist()}
    - Salary distribution by ACTUAL location and job function
    - Current geographic distribution IN THE PROVIDED DATA
    - DO NOT invent locations outside: {df['location'].unique().tolist()}""",
    expected_output="A report using ONLY the provided locations and data.",
    agent=data_analyst,
    output_file="workforce_analysis.md"
)




# Task 2: Propose cost reductions
cost_reduction_task = Task(
    description="""Develop a proposal to reduce salary costs by 20% while:
    - Minimizing layoffs
    - Considering salary adjustments
    - Identifying potential role consolidations
    Provide specific recommendations with justifications""",
    expected_output="A detailed cost reduction plan with specific actions and expected savings.",
    agent=cost_optimizer,
    context=[analysis_task]
)

# Task 3: Validate geographic balance
geo_balance_task = Task(
    description="""Review the cost reduction proposal and ensure it:
    - Maintains minimum presence in each location
    - Doesn't disproportionately impact any single region
    - Considers local labor market conditions
    Provide modifications if needed""",
    expected_output="An approved or modified cost reduction plan that maintains geographic balance.",
    agent=geo_specialist,
    context=[cost_reduction_task]
)

# Task 4: Validate job function balance
role_balance_task = Task(
    description="""Review the cost reduction proposal and ensure it:
    - Maintains critical functions in each location
    - Preserves necessary skill sets
    - Doesn't create single points of failure
    Provide modifications if needed""",
    expected_output="An approved or modified cost reduction plan that maintains job function balance.",
    agent=role_specialist,
    context=[geo_balance_task]
)

In [ ]:
# Form the crew
salary_optimization_crew = Crew(
    agents=[data_analyst, cost_optimizer, geo_specialist, role_specialist],
    tasks=[analysis_task, cost_reduction_task, geo_balance_task, role_balance_task],
    verbose=True
)

# Execute the crew
results = salary_optimization_crew.kickoff(inputs={'employee_data': df.to_dict()})

# Print results
print("\n\nFinal Optimization Plan:")
print(results)

In [ ]:
import pandas as pd
from crewai import Agent, Task, Crew

# Test basic functionality
def test_environment():
    data = {'test': [10, 20, 30]}
    df = pd.DataFrame(data)
    print("Pandas test:", df.head())
    
    try:
        agent = Agent(
            role='test',
            goal='Test the environment',
            backstory='Just testing',
            allow_delegation=False
        )
        print("CrewAI test: Agent created successfully")
    except Exception as e:
        print("CrewAI test failed:", e)

if __name__ == "__main__":
    test_environment()